# MIRACL Dataset
This Notebook demonstrates the setup of the MIRACL dataset for use in _PyTerrier_. The dataset is available on Huggingface and comprises two parts:

1. **[miracl/miracl-corpus](https://huggingface.co/datasets/miracl/miracl-corpus)**: Contains the _corpus_ data.

2. **[miracl/miracl](https://huggingface.co/datasets/miracl/miracl)**: Contains the _topics_ and _qrels_.


In [23]:
# Dependencies
# %pip install python-terrier   # PyTerrier
# %pip install datasets         # Hugging Face
# %pip install tqdm             # tqdm progress bars
# %pip install pandas           # pandas

# somehow necessary on Mac, not sure about other OS
# %pip install --upgrade jupyter ipywidgets

In [24]:
# Libraries
import pyterrier as pt
import datasets
import pandas as pd
from tqdm import tqdm

In [25]:
# Initialize PyTerrier
if not pt.started():
    pt.init()

In [26]:
# Load miracl datasets
lang='sw'   # choose language
miracl_corpus = datasets.load_dataset('miracl/miracl-corpus', lang, trust_remote_code=True) # splits: train
miracl_queries = datasets.load_dataset('miracl/miracl', lang, trust_remote_code=True)       # splits: train, dev, testA, testB

In [27]:
# # Limit amount of docs and queries (for testing with large datasets)
# def limited_miracl_corpus_iter(limit=1000):
#     count = 0
#     for doc in tqdm(miracl_corpus['train'], desc="Processing Corpus"):
#         if count >= limit:
#             break
#         yield {
#             'docno': doc['docid'], 
#             'title': doc['title'],
#             'text': doc['text']
#         }
#         count += 1

# # Preparing queries and qrels for PyTerrier
# queries = []
# qrels = []
# for idx, data in enumerate(tqdm(miracl_queries[split], desc="Processing Queries and Qrels")):
#     if idx >= 10:  # Limit to first 10 queries
#         break
#     queries.append({'qid': data['query_id'], 'query': data['query']})
#     for entry in data['positive_passages']:
#         qrels.append({'qid': data['query_id'], 'docno': entry['docid'], 'label': 1})
#     for entry in data['negative_passages']:
#         qrels.append({'qid': data['query_id'], 'docno': entry['docid'], 'label': 0})
#
# queries_df = pd.DataFrame(queries)
# qrels_df = pd.DataFrame(qrels)


In [28]:
# Choose split for queries dataset
split = 'train'   # 'dev', 'train', 'testA', 'testB'

# Corpus iterator
def miracl_corpus_iter():
    for doc in tqdm(miracl_corpus['train'], desc="Processing Corpus"):
        yield {
            'docno': doc['docid'], 
            'title': doc['title'],
            'text': doc['text']
        }

# Preparing queries and qrels for PyTerrier
queries = []
qrels = []
for data in tqdm(miracl_queries[split], desc="Processing Queries and Qrels"):
    queries.append({'qid': data['query_id'], 'query': data['query']})
    for entry in data['positive_passages']:
        qrels.append({'qid': data['query_id'], 'docno': entry['docid'], 'label': 1})
    for entry in data['negative_passages']:
        qrels.append({'qid': data['query_id'], 'docno': entry['docid'], 'label': 0})


queries_df = pd.DataFrame(queries)
qrels_df = pd.DataFrame(qrels)

Processing Queries and Qrels: 100%|██████████| 1901/1901 [00:00<00:00, 28588.95it/s]


In [29]:
# Indexing
indexer = pt.IterDictIndexer("./miracl_index", overwrite=True, blocks=True)
index_ref = indexer.index(miracl_corpus_iter())

Processing Corpus:   0%|          | 0/131924 [00:00<?, ?it/s]

09:55:01.699 [ForkJoinPool-3-worker-1] WARN org.terrier.structures.indexing.Indexer - Adding an empty document to the index (30#5) - further warnings are suppressed


Processing Corpus: 100%|██████████| 131924/131924 [00:04<00:00, 29884.61it/s]


09:55:07.482 [ForkJoinPool-3-worker-1] WARN org.terrier.structures.indexing.Indexer - Indexed 129 empty documents


In [30]:
# BM25 Single Query Testing/Debugging
# bm25 = pt.BatchRetrieve(index_ref, wmodel="BM25")
# test_query_df = pd.DataFrame([{'qid': 'test', 'query': "Testx"}])
# test_query_df['query'] = test_query_df['query'].str.replace("x", "omato")
# print(test_query_df)
# test_results = bm25.transform(test_query_df)

In [31]:
# BM25
bm25 = pt.BatchRetrieve(index_ref, wmodel="BM25")

# Preprocessing
queries_df['query'] = queries_df['query'].str.replace('?', '')  # remove question marks
queries_df['query'] = queries_df['query'].str.replace("'", "")  # remove apostrophes
queries_df['query'] = queries_df['query'].str.replace("/", "")  # remove slash
queries_df['query'] = queries_df['query'].str.replace("!", "")  # remove exclamation mark
# add further replace statements should you encounter any "Lexical Error"

# Apply BM25 to preprocessed queries
results = bm25.transform(queries_df)

In [32]:
# Evaluation
eval_metrics = ['map', 'ndcg']
eval_results = pt.Experiment(
    [bm25],
    queries_df,
    qrels_df,
    eval_metrics=eval_metrics,
    #perquery=True
)

print(eval_results)

       name       map      ndcg
0  BR(BM25)  0.211017  0.325033
